In [8]:
from ultralytics import YOLO
import cv2
from supervision.utils.video import VideoInfo, VideoSink, get_video_frames_generator
from supervision.utils.notebook import plot_image
from supervision.detection.core import Detections
from supervision.detection.annotate import BoxAnnotator
from supervision.draw.color import ColorPalette, Color
from tqdm.notebook import tqdm
from PIL import ImageColor

In [9]:
model = YOLO('../models/yolov8n.pt')
model.fuse()

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [13]:
VIDEO_PATH = '../sources/traffic_1.mp4'
TARGET_PATH = '../outputs/traffic_detected_1.avi'

In [10]:
CLASS_NAMES_DICT = model.model.names
colors = [
    "#a351fb", "#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6",
    "#d2f53c", "#fabebe", "#008080", "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3",
]
colors = [ImageColor.getcolor(color, "RGB") for color in colors]
COLORS = [Color(r, g, b) for r, g, b in colors]

In [14]:
generator = get_video_frames_generator(VIDEO_PATH)
box_annotator = BoxAnnotator(color=ColorPalette(COLORS), thickness=4, text_thickness=1, text_scale=1)
video_info = VideoInfo.from_video_path(VIDEO_PATH)

with VideoSink(TARGET_PATH, video_info) as sink:
    for frame in tqdm(generator, total=video_info.total_frames):
        results = model(frame)

        detections = Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int)
        )
        labels = [
            f'{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}'
            for _, _, confidence, class_id, _,
            in detections
        ]

        frame = box_annotator.annotate(frame, detections=detections, labels=labels)

        sink.write_frame(frame)

  0%|          | 0/3600 [00:00<?, ?it/s]

0: 384x640 20 cars, 1 bus, 2 trucks, 1598.2ms
Speed: 442.7ms preprocess, 1598.2ms inference, 805.8ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 20 cars, 1 bus, 2 trucks, 700.1ms
Speed: 13.0ms preprocess, 700.1ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 20 cars, 1 bus, 3 trucks, 348.1ms
Speed: 18.0ms preprocess, 348.1ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 21 cars, 1 bus, 2 trucks, 221.4ms
Speed: 10.0ms preprocess, 221.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 22 cars, 1 bus, 2 trucks, 277.3ms
Speed: 7.0ms preprocess, 277.3ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 22 cars, 1 bus, 2 trucks, 389.0ms
Speed: 10.0ms preprocess, 389.0ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 22 cars, 1 bus, 2 trucks, 593.4ms
Speed: 17.9ms preprocess, 593.4ms inference, 6.0ms postprocess per image at shape (1, 3

In [4]:
import yaml
from pathlib import Path
conf = yaml.safe_load(open('../variables/variables.yaml', 'r'))

In [7]:
conf['MODELS']

['yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt']

In [9]:
from ultralytics import YOLO
model = YOLO('../models/yolov8n.pt')

In [10]:
result = model.predict('../sources/traffic_1.mp4', show=True, save=True)


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs
video 1/1 (1/3600) C:\Users\DELL\OneDrive\Desktop\AI\Object_Detection\research\..\sources\traffic_1.mp4: 384x640 20 cars, 1 bus, 2 trucks, 215.5ms
video 1/1 (2/3600) C:\Users\DELL\OneDrive\Desktop\AI\Object_Detection\research\..\sources\traffic_1.mp4: 384x640 20 cars, 1 bus, 2 trucks, 690.2ms
video 1/1 (3/3600) C:\Users\DELL\OneDrive\Desktop\AI\Object_Detection\research\..\sources\traffic_1.mp4: 384x640 20 cars, 1 bus, 3 trucks, 312.0ms
video 1/1 (4/3600

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\alloc.cpp:73: error: (-4:Insufficient memory) Failed to allocate 6220800 bytes in function 'cv::OutOfMemoryError'


In [7]:
from ultralytics import YOLO
import cv2

model = YOLO('../models/yolov8n.pt')
video = cv2.VideoCapture(0)  # Use 0 for the default camera

frame_width = int(video.get(3))
frame_height = int(video.get(4))

size = (frame_width, frame_height)

output = cv2.VideoWriter('../outputs/from_webcam/',
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

while True:
    ret, img0 = video.read()

    if ret == True:
        result = model(img0)
        frame = result[0].plot()
        cv2.imshow('Frame', frame)
        output.write(frame)
        key = cv2.waitKey(1)
        if key == 27:  # ASCII code for "Esc" key
            break
    else:
        break

# Release the webcam and close any open windows
video.release()
cv2.destroyAllWindows()

0: 480x640 1 cat, 240.4ms
Speed: 8.0ms preprocess, 240.4ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 341.1ms
Speed: 20.9ms preprocess, 341.1ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 245.3ms
Speed: 6.0ms preprocess, 245.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 1 cat, 200.5ms
Speed: 4.0ms preprocess, 200.5ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 cat, 255.3ms
Speed: 4.0ms preprocess, 255.3ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 cat, 213.5ms
Speed: 4.0ms preprocess, 213.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 1 dog, 187.5ms
Speed: 4.0ms preprocess, 187.5ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 204.5ms
Speed: 4.0ms preprocess, 204.5ms inference, 3.0ms postprocess per image at s

In [12]:
from utils import load_yaml_as_dict
load_yaml_as_dict('../variables/variables.yaml')

{'COLORS': ['#a351fb',
  '#e6194b',
  '#3cb44b',
  '#ffe119',
  '#0082c8',
  '#f58231',
  '#911eb4',
  '#46f0f0',
  '#f032e6',
  '#d2f53c',
  '#fabebe',
  '#008080',
  '#e6beff',
  '#aa6e28',
  '#fffac8',
  '#800000',
  '#aaffc3'],
 'PATHS': [{'VIDEO_PATH': 'sources/'},
  {'TARGET_PATH': 'outputs/'},
  {'MODELS_PATH': 'models/'}],
 'MODELS': ['yolov8n.pt',
  'yolov8s.pt',
  'yolov8m.pt',
  'yolov8l.pt',
  'yolov8x.pt']}

In [14]:
colors = load_yaml_as_dict('../variables/variables.yaml')['PATHS']
colors

[{'VIDEO_PATH': 'sources/'},
 {'TARGET_PATH': 'outputs/'},
 {'MODELS_PATH': 'models/'}]

In [15]:
colors[0]['VIDEO_PATH']

'sources/'

In [1]:
string = 'I met my nominee as a classmate. I engaged with him in curricular and extra-curricular activities like group studies, tech seminars, webinars, tech communities adn more.'

In [4]:
len(string.split())

26

In [3]:
len(string)

169